<a href="https://colab.research.google.com/github/SrimaniGabrielle/16.09.2022/blob/main/Fitfoodies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 74.0 MB/s eta 0:00:00


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pulp import *
import seaborn as sns
import random

In [4]:
data = pd.read_csv('nutrition.csv').drop('Unnamed: 0',axis=1)
data.head()

,name,serving_size,calories,total_fat,saturated_fat,cholesterol,sodium,choline,folate,folic_acid,...,fat,saturated_fatty_acids,monounsaturated_fatty_acids,polyunsaturated_fatty_acids,fatty_acids_total_trans,alcohol,ash,caffeine,theobromine,water
0,Cornstarch,100 g,381,0.1g,NaN,0,9.00 mg,0.4 mg,0.00 mcg,0.00 mcg,...,0.05 g,0.009 g,0.016 g,0.025 g,0.00 mg,0.0 g,0.09 g,0.00 mg,0.00 mg,8.32 g
1,"Nuts, pecans",100 g,691,72g,6.2g,0,0.00 mg,40.5 mg,22.00 mcg,0.00 mcg,...,71.97 g,6.180 g,40.801 g,21.614 g,0.00 mg,0.0 g,1.49 g,0.00 mg,0.00 mg,3.52 g
2,"Eggplant, raw",100 g,25,0.2g,NaN,0,2.00 mg,6.9 mg,22.00 mcg,0.00 mcg,...,0.18 g,0.034 g,0.016 g,0.076 g,0.00 mg,0.0 g,0.66 g,0.00 mg,0.00 mg,92.30 g
3,"Teff, uncooked",100 g,367,2.4g,0.4g,0,12.00 mg,13.1 mg,0,0,...,2.38 g,0.449 g,0.589 g,1.071 g,0,0,2.37 g,0,0,8.82 g
4,"Sherbet, orange",100 g,144,2g,1.2g,1mg,46.00 mg,7.7 mg,4.00 mcg,0.00 mcg,...,2.00 g,1.160 g,0.530 g,0.080 g,1.00 mg,0.0 g,0.40 g,0.00 mg,0.00 mg,66.10 g


# New Section

In [5]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8789 entries, 0 to 8788
Data columns (total 76 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   name                         8789 non-null   object
 1   serving_size                 8789 non-null   object
 2   calories                     8789 non-null   int64 
 3   total_fat                    8789 non-null   object
 4   saturated_fat                7199 non-null   object
 5   cholesterol                  8789 non-null   object
 6   sodium                       8789 non-null   object
 7   choline                      8789 non-null   object
 8   folate                       8789 non-null   object
 9   folic_acid                   8789 non-null   object
 10  niacin                       8789 non-null   object
 11  pantothenic_acid             8789 non-null   object
 12  riboflavin                   8789 non-null   object
 13  thiamin                      8789

In [6]:
data = data.drop('serving_size',axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8789 entries, 0 to 8788
Data columns (total 75 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   name                         8789 non-null   object
 1   calories                     8789 non-null   int64 
 2   total_fat                    8789 non-null   object
 3   saturated_fat                7199 non-null   object
 4   cholesterol                  8789 non-null   object
 5   sodium                       8789 non-null   object
 6   choline                      8789 non-null   object
 7   folate                       8789 non-null   object
 8   folic_acid                   8789 non-null   object
 9   niacin                       8789 non-null   object
 10  pantothenic_acid             8789 non-null   object
 11  riboflavin                   8789 non-null   object
 12  thiamin                      8789 non-null   object
 13  vitamin_a                    8789

In [7]:
data['carbohydrate'] = np.array([data['carbohydrate'].tolist()[i].split(' ') for i in range(len(data))])[:,0].astype('float')
data['protein'] = np.array([data['protein'].tolist()[i].split(' ') for i in range(len(data))])[:,0].astype('float')
data['total_fat'] = np.array([data['total_fat'].tolist()[i].split('g') for i in range(len(data))])[:,0].astype('float')

In [8]:
week_days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
split_values = np.linspace(0,len(data),8).astype(int)
split_values[-1] = split_values[-1]-1
def random_dataset():
    frac_data = data.sample(frac=1).reset_index().drop('index',axis=1)
    day_data = []
    for s in range(len(split_values)-1):
        day_data.append(frac_data.loc[split_values[s]:split_values[s+1]])
    return dict(zip(week_days,day_data))

In [9]:
random_dataset()['Monday']

,name,calories,total_fat,saturated_fat,cholesterol,sodium,choline,folate,folic_acid,niacin,...,fat,saturated_fatty_acids,monounsaturated_fatty_acids,polyunsaturated_fatty_acids,fatty_acids_total_trans,alcohol,ash,caffeine,theobromine,water
0,"Fish, dry heat, cooked, spanish, mackerel",158,6.3,1.8g,73mg,66.00 mg,0,1.00 mcg,0.00 mcg,5.000 mg,...,6.32 g,1.801 g,2.139 g,1.805 g,73.00 mg,0,1.53 g,0,0,68.46 g
1,"Pork, raw, separable lean and fat, blade (stea...",186,12.0,4.3g,62mg,61.00 mg,73.3 mg,0.00 mcg,0.00 mcg,4.239 mg,...,12.36 g,4.348 g,5.206 g,1.597 g,62.00 mg,0.0 g,0.83 g,0.00 mg,0.00 mg,69.18 g
2,"Eppaw, raw",150,1.8,NaN,0,12.00 mg,0,24.00 mcg,0.00 mcg,0.300 mg,...,1.80 g,0,0,0,0.00 mg,0,1.92 g,0,0,60.00 g
3,"Cheese food, without added vitamin D, American...",330,26.0,15g,98mg,1441.00 mg,29.9 mg,7.00 mcg,0.00 mcg,0.155 mg,...,25.63 g,15.057 g,6.846 g,1.049 g,98.00 mg,0.0 g,4.96 g,0.00 mg,0.00 mg,44.00 g
4,"Waxgourd, with salt, drained, boiled, cooked, ...",11,0.2,NaN,0,343.00 mg,4.9 mg,4.00 mcg,0.00 mcg,0.384 mg,...,0.20 g,0.016 g,0.037 g,0.088 g,0.00 mg,0,0.89 g,0,0,96.06 g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1251,"Soybeans, raw, sprouted, mature seeds",122,6.7,0.9g,0,14.00 mg,0,172.00 mcg,0.00 mcg,1.148 mg,...,6.70 g,0.929 g,1.518 g,3.783 g,0.00 mg,0,1.59 g,0,0,69.05 g
1252,"Chicken, stewed, cooked, meat and skin, leg, b...",220,13.0,3.6g,84mg,73.00 mg,56.4 mg,6.00 mcg,0.00 mcg,4.590 mg,...,12.92 g,3.570 g,5.040 g,2.870 g,84.00 mg,0.0 g,0.79 g,0.00 mg,0.00 mg,64.01 g
1253,"Mayonnaise, cholesterol-free, reduced-calorie ...",333,33.0,4.6g,0,733.00 mg,7.0 mg,0.00 mcg,0.00 mcg,0.000 mg,...,33.30 g,4.618 g,7.707 g,19.500 g,0.00 mg,0.0 g,3.10 g,0.00 mg,0.00 mg,56.00 g
1254,"Cereals, without salt, prepared (microwaved or...",74,0.5,0.1g,0,227.00 mg,2.6 mg,27.00 mcg,15.00 mcg,1.672 mg,...,0.49 g,0.076 g,0.081 g,0.191 g,0,0,0.76 g,0,0,81.22 g


In [10]:
def build_nutritional_values(kg,calories):
    protein_calories = kg*4
    res_calories = calories-protein_calories
    carb_calories = calories/2.
    fat_calories = calories-carb_calories-protein_calories
    res = {'Protein Calories':protein_calories,'Carbohydrates Calories':carb_calories,'Fat Calories':fat_calories}
    return res

In [11]:
def extract_gram(table):
    protein_grams = table['Protein Calories']/4.
    carbs_grams = table['Carbohydrates Calories']/4.
    fat_grams = table['Fat Calories']/9.
    res = {'Protein Grams':protein_grams, 'Carbohydrates Grams':carbs_grams,'Fat Grams':fat_grams}
    return res

In [12]:
build_nutritional_values(70,2000)

{'Protein Calories': 280,
 'Carbohydrates Calories': 1000.0,
 'Fat Calories': 720.0}

In [13]:
print(extract_gram(build_nutritional_values(70,2000)))

{'Protein Grams': 70.0, 'Carbohydrates Grams': 250.0, 'Fat Grams': 80.0}


In [14]:
days_data = random_dataset()
def model(day,kg,calories):
    G = extract_gram(build_nutritional_values(kg,calories))
    E = G['Carbohydrates Grams']
    F = G['Fat Grams']
    P = G['Protein Grams']
    day_data = days_data[day]
    day_data = day_data[day_data.calories!=0]
    food = day_data.name.tolist()
    c  = day_data.calories.tolist()
    x  = pulp.LpVariable.dicts( "x", indices = food, lowBound=0, upBound=1.5, cat='Continuous', indexStart=[] )
    e = day_data.carbohydrate.tolist()
    f = day_data.total_fat.tolist()
    p = day_data.protein.tolist()
    prob  = pulp.LpProblem( "Diet", LpMinimize )
    prob += pulp.lpSum( [x[food[i]]*c[i] for i in range(len(food))]  )
    prob += pulp.lpSum( [x[food[i]]*e[i] for i in range(len(x)) ] )>=E
    prob += pulp.lpSum( [x[food[i]]*f[i] for i in range(len(x)) ] )>=F
    prob += pulp.lpSum( [x[food[i]]*p[i] for i in range(len(x)) ] )>=P
    prob.solve()
    variables = []
    values = []
    for v in prob.variables():
        variable = v.name
        value = v.varValue
        variables.append(variable)
        values.append(value)
    values = np.array(values).round(2).astype(float)
    sol = pd.DataFrame(np.array([food,values]).T, columns = ['Food','Quantity'])
    sol['Quantity'] = sol.Quantity.astype(float)
    return sol

In [15]:
sol_monday = model('Monday',70,1500)

In [16]:
sol_monday = sol_monday[sol_monday['Quantity']!=0.0]
sol_monday.Quantity = sol_monday.Quantity*100
sol_monday = sol_monday.rename(columns={'Quantity':'Quantity (g)'})
sol_monday

,Food,Quantity (g)
58,"Tortellini, as purchased, fresh-refrigerated, ...",50.0
272,"Crustaceans, moist heat, cooked, dungeness, crab",84.0
474,"Biscuits, prepared, dry mix, plain or buttermilk",150.0
1010,"Restaurant, chili with meat and beans, family ...",50.0
1123,"Pineapple juice, undiluted, unsweetened, froze...",150.0
1124,"Rice flour, unenriched, white",150.0


In [17]:
def model(prob,day,kg,calories):
    G = extract_gram(build_nutritional_values(kg,calories))
    E = G['Carbohydrates Grams']
    F = G['Fat Grams']
    P = G['Protein Grams']
    day_data = days_data[day]
    day_data = day_data[day_data.calories!=0]
    food = day_data.name.tolist()
    c  = day_data.calories.tolist()
    x  = pulp.LpVariable.dicts( "x", indices = food, lowBound=0, upBound=1.5, cat='Continuous', indexStart=[] )
    e = day_data.carbohydrate.tolist()
    f = day_data.total_fat.tolist()
    p = day_data.protein.tolist()
#    prob  = pulp.LpProblem( "Diet", LpMinimize )
    prob += pulp.lpSum( [x[food[i]]*c[i] for i in range(len(food))]  )
    prob += pulp.lpSum( [x[food[i]]*e[i] for i in range(len(x)) ] )>=E
    prob += pulp.lpSum( [x[food[i]]*f[i] for i in range(len(x)) ] )>=F
    prob += pulp.lpSum( [x[food[i]]*p[i] for i in range(len(x)) ] )>=P
    prob.solve()
    variables = []
    values = []
    for v in prob.variables():
        variable = v.name
        value = v.varValue
        variables.append(variable)
        values.append(value)
    values = np.array(values).round(2).astype(float)
    sol = pd.DataFrame(np.array([food,values]).T, columns = ['Food','Quantity'])
    sol['Quantity'] = sol.Quantity.astype(float)
    sol = sol[sol['Quantity']!=0.0]
    sol.Quantity = sol.Quantity*100
    sol = sol.rename(columns={'Quantity':'Quantity (g)'})
    return sol
def total_model(kg,calories):
    result = []
    for day in week_days:
        prob  = pulp.LpProblem( "Diet", LpMinimize )
        print('Building a model for day %s \n'%(day))
        result.append(model(prob,day,kg,calories))
    return dict(zip(week_days,result))

In [18]:
diet = total_model(70,3000)

Building a model for day Monday 

Building a model for day Tuesday 

Building a model for day Wednesday 

Building a model for day Thursday 

Building a model for day Friday 

Building a model for day Saturday 

Building a model for day Sunday 



In [19]:
diet

{'Monday':                                                    Food  Quantity (g)
 58    Tortellini, as purchased, fresh-refrigerated, ...         150.0
 272    Crustaceans, moist heat, cooked, dungeness, crab         150.0
 401   Pork, broiled, cooked, separable lean only, bo...         150.0
 474    Biscuits, prepared, dry mix, plain or buttermilk         150.0
 768          Babyfood, with calcium, fruit punch, juice           9.0
 843                       Pulled pork in barbecue sauce          61.0
 917    Cookies, lowfat, plain or honey, graham crackers         150.0
 1123  Pineapple juice, undiluted, unsweetened, froze...          63.0,
 'Tuesday':                                                    Food  Quantity (g)
 253   Lamb, braised, cooked, trimmed to 1/8" fat, se...         150.0
 383   Oil, used for whipped toppings and coffee whit...         150.0
 461                           Lupins, raw, mature seeds         150.0
 763                                 Game meat, raw, el

In [20]:
split_size = np.linspace(0,len(data),8)

In [23]:
diet['Monday']

,Food,Quantity (g)
58,"Tortellini, as purchased, fresh-refrigerated, ...",150.0
272,"Crustaceans, moist heat, cooked, dungeness, crab",150.0
401,"Pork, broiled, cooked, separable lean only, bo...",150.0
474,"Biscuits, prepared, dry mix, plain or buttermilk",150.0
768,"Babyfood, with calcium, fruit punch, juice",9.0
843,Pulled pork in barbecue sauce,61.0
917,"Cookies, lowfat, plain or honey, graham crackers",150.0
1123,"Pineapple juice, undiluted, unsweetened, froze...",63.0
